In [ ]:
import os
import re
import git
import sys
import glob
import json
import joblib
import sklearn
import skimage
import tifffile
import imageio
import numpy as np
import pandas as pd
import seaborn as sns
import pathlib

import matplotlib
from matplotlib import pyplot as plt

In [ ]:
from dragonfly_automation import gateway_utils, microscope_operations
from dragonfly_automation.acquisitions.pipeline_plate_settings import (
    StackSettings,
    dev_fluorescence_stack_settings
)
from dragonfly_automation.tests.mocks import mock_gateway
from dragonfly_automation.fov_models import PipelineFOVScorer

In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
gate = mock_gateway.Gate('random-real')
mm_core = gate.getCMMCore()
mm_studio = gate.getStudio()

datastore = mm_studio.data().createMultipageTIFFDatastore()

messages = []
def event_logger(message):
    print(message)

stack_settings = StackSettings(
    stage_label='label', relative_bottom=0, relative_top=3, step_size=1
)

In [ ]:
mm_core._throw_get_tagged_image_error = True

In [ ]:
microscope_operations.acquire_stack(
    mm_studio, 
    mm_core, 
    datastore, 
    stack_settings, 
    channel_ind=0,
    position_ind=123,
    position_name='position_name',
    event_logger=event_logger
)

In [ ]:
[str(image.getCoords()) for image in datastore.images]

### Manually select FOVs from existing datasets

In [ ]:
fov_scorer = PipelineFOVScorer(save_dir='../models/2019-10-08/', mode='prediction')
fov_scorer.load()
fov_scorer.train()
fov_scorer.validate()

In [ ]:
# PML0216 had progressively less clumpiness, with some too-few and no-nuclei FOVs
pml_dir = pathlib.Path('/Volumes/ml_group/raw-pipeline-microscopy/PML0216/')

# PML0316 was a very good full-plate
# pml_dir = pathlib.Path('/Volumes/ml_group/raw-pipeline-microscopy/PML0316/')

In [ ]:
snaps_dir = pml_dir / 'logs/fov-scoring/fov-images/'

df = pd.read_csv(pml_dir / 'logs/fov-scoring/fov-score-log.csv')

scored = df.loc[df.score.notna()]
too_few = df.loc[df.comment == 'Too few nuclei in the FOV']
no_nuclei = df.loc[df.comment == 'No nuclei in the FOV']

scored = scored.sort_values(by='score', ascending=False)

In [ ]:
scored.shape, too_few.shape, no_nuclei.shape

In [ ]:
row = too_few.iloc[0]
filepath = snaps_dir / f'FOV_{row.position_name}_RAW.tif'
im = imageio.imread(filepath)
plt.imshow(im)

result = fov_scorer.score_raw_fov(im, min_otsu_thresh=700, min_num_nuclei=10)
row.score, result.get('score'), result.get('comment')

In [ ]:
dst_dirpath = '/Users/keith.cheveralls/projects/dragonfly-automation/dragonfly_automation/tests/artifacts/snaps/'
import shutil
shutil.copy2(filepath, os.path.join(dst_dirpath, 'too-few-1.tif'))